In [ ]:
!pip3 install statsmodels
import pandas as pd
import numpy as np
import requests
import statsmodels.api as sm
import matplotlib.pyplot as plt

import os

# Define API key and function to fetch OHLC data from Polygon.io
API_KEY = os.getenv('POLYGON_API_KEY')

def fetch_ohlc(symbol, timespan="minute", multiplier=1, limit=1000):
    url = f"https://api.polygon.io/v2/aggs/ticker/{symbol}/range/{multiplier}/{timespan}/2023-01-01/2023-12-31?adjusted=true&sort=asc&limit={limit}&apiKey={API_KEY}"
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data['results'])
    df['timestamp'] = pd.to_datetime(df['t'], unit='ms')
    df.set_index('timestamp', inplace=True)
    return df[['o', 'h', 'l', 'c', 'v']]

# Step 1: Initialize trend using linear regression on the first 30 candles
def initialize_trend(df):
    df_init = df.iloc[:30]
    X = np.arange(30).reshape(-1, 1)
    y = df_init['c'].values
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    trend_slope = model.params[1]
    return "uptrend" if trend_slope > 0 else "downtrend"

# Step 2: Identify valid high and low
class Trend:
    def __init__(self, df):
        self.df = df
        self.trend = initialize_trend(df)
        self.valid_high = df['h'][:30].max()
        self.valid_low = df['l'][:30].min()
        self.swings = []

    def update_trend(self, idx):
        if self.trend == "uptrend" and self.df['c'].iloc[idx] < self.valid_low:
            self.trend = "downtrend"
            self.valid_high = self.df['h'].iloc[idx]
        elif self.trend == "downtrend" and self.df['c'].iloc[idx] > self.valid_high:
            self.trend = "uptrend"
            self.valid_low = self.df['l'].iloc[idx]

    def check_swings(self, idx):
        if self.trend == "uptrend" and self.df['l'].iloc[idx] > self.valid_low:
            self.swings.append((self.df.index[idx], self.df['l'].iloc[idx]))
        elif self.trend == "downtrend" and self.df['h'].iloc[idx] < self.valid_high:
            self.swings.append((self.df.index[idx], self.df['h'].iloc[idx]))

# Step 3: Backtesting function
def backtest(df):
    trend_obj = Trend(df)
    balance = 10000  # Starting balance
    position = None
    for i in range(30, len(df)):
        trend_obj.update_trend(i)
        trend_obj.check_swings(i)
        
        if position is None and trend_obj.swings:
            swing = trend_obj.swings[-1]
            entry_price = swing[1] * 1.05 if trend_obj.trend == "uptrend" else swing[1] * 0.95
            stop_loss = trend_obj.valid_low if trend_obj.trend == "uptrend" else trend_obj.valid_high
            take_profit = trend_obj.valid_high if trend_obj.trend == "uptrend" else trend_obj.valid_low
            
            position = {'entry': entry_price, 'stop_loss': stop_loss, 'take_profit': take_profit}
        
        if position:
            if df['h'].iloc[i] >= position['take_profit']:
                balance *= 1.1
                position = None
            elif df['l'].iloc[i] <= position['stop_loss']:
                balance *= 0.9
                position = None
    return balance

# Fetch data and run backtest
df = fetch_ohlc("AAPL", timespan="day", multiplier=1, limit=500)
final_balance = backtest(df)
print(f"Final Balance: ${final_balance:.2f}")


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 9.9 MB 922 kB/s eta 0:00:01
     |████████████████████████████████| 232 kB 13.1 MB/s eta 0:00:01
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Final Balance: $233290985735.63
